TODO: change to use own model instead of pretrained

In [20]:
from pyannote.audio import Pipeline
import os
from dotenv import load_dotenv
import re
from pydub import AudioSegment

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

# TODO: change to own model
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=huggingface_token)

C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
path = "../training_database/wavs/file19.wav"

In [ ]:
# apply pretrained pipeline
diarization = pipeline(path)    

create rttm file based on model result

In [ ]:
file_name = re.search(r'.*\/(.+?)\.wav', path).group(1)

f = open(f"./results/rttm/{file_name}.rttm", "w")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    f.write(f"SPEAKER {file_name} 1 {round(turn.start, 3)} {round(turn.end-turn.start, 3)} <NA> <NA> {speaker} <NA> <NA>\n")

f.close()

turn result into audio files

In [30]:
audio = AudioSegment.from_wav(f"../training_database/wavs/{file_name}.wav")
os.mkdir(f"./results/audio/{file_name}/")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    if not os.path.isdir(f"./results/audio/{file_name}/{speaker}"):
        os.mkdir(f"./results/audio/{file_name}/{speaker}")

    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./results/audio/{file_name}/{speaker}/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")

create audio file based on speaker by combining audio files

In [ ]:
speakers = os.listdir(f"./results/audio/{file_name}")

for speaker in speakers:
    files_list = os.listdir(f"./results/audio/{file_name}/{speaker}")

    combined = AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[0]}")
    for i in range(1,len(files_list)):
        combined = combined + AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[i]}")

    combined.export(f"./results/audio/{file_name}_{speaker}.wav", format="wav")

['speaker_SPEAKER_00_127.0-128.4.wav', 'speaker_SPEAKER_00_128.5-128.5.wav', 'speaker_SPEAKER_00_129.4-133.2.wav', 'speaker_SPEAKER_00_77.5-77.5.wav']
['speaker_SPEAKER_01_10.8-12.4.wav', 'speaker_SPEAKER_01_103.4-104.7.wav', 'speaker_SPEAKER_01_106.7-113.1.wav', 'speaker_SPEAKER_01_113.7-121.2.wav', 'speaker_SPEAKER_01_12.7-14.8.wav', 'speaker_SPEAKER_01_122.0-124.9.wav', 'speaker_SPEAKER_01_128.4-129.3.wav', 'speaker_SPEAKER_01_135.2-136.0.wav', 'speaker_SPEAKER_01_139.0-139.7.wav', 'speaker_SPEAKER_01_140.7-142.0.wav', 'speaker_SPEAKER_01_15.4-18.3.wav', 'speaker_SPEAKER_01_18.6-19.3.wav', 'speaker_SPEAKER_01_21.0-21.7.wav', 'speaker_SPEAKER_01_22.6-26.0.wav', 'speaker_SPEAKER_01_26.5-28.6.wav', 'speaker_SPEAKER_01_28.8-30.6.wav', 'speaker_SPEAKER_01_31.2-34.9.wav', 'speaker_SPEAKER_01_34.9-37.5.wav', 'speaker_SPEAKER_01_38.0-38.8.wav', 'speaker_SPEAKER_01_41.6-43.5.wav', 'speaker_SPEAKER_01_44.0-48.0.wav', 'speaker_SPEAKER_01_49.0-51.3.wav', 'speaker_SPEAKER_01_52.2-59.6.wav', 'spe